# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Mentoría 16 - ¿Cómo identificar fuga de ventas? Inteligencia artificial aplicada al sector comercial.

### Explorando Patrones de Datos a través de Clustering (TP3)

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---
## Librerías

In [ ]:
# Para que las funciones se actualicen sin tener que refrescar el kernel
%load_ext autoreload
%autoreload 2

# Funciones de visualización y curación
import pandas as pd
import json
from os.path import exists
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress as LR
from scipy.stats import skew, kurtosis, skewtest, kurtosistest

# Funciones de clustering
from sklearn import manifold, preprocessing, decomposition
from sklearn.cluster import KMeans, MeanShift
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# Funciones propias
from tp2_utils_limpieza import * 

# Clear preferencias
plt.rcdefaults()
pd.reset_option('^display\.float_format')

---
# Preparación de Datos <span style="color:magenta">**(Paso 1)**</span>

In [ ]:
vectores = pd.read_csv('../data/interim/tp3_Fven_limpio_ceros.csv')
vectores

In [ ]:
# Definir grupos de variables:

F_Com = ['F_pct_Com_1905', 'F_pct_Com_1906', 'F_pct_Com_1907', 'F_pct_Com_1908', 
         'F_pct_Com_1909', 'F_pct_Com_1910', 'F_pct_Com_1911', 'F_pct_Com_1912',
         'F_pct_Com_2001', 'F_pct_Com_2002', 'F_pct_Com_2003', 'F_pct_Com_2004',
         'F_pct_Com_2005', 'F_pct_Com_2006', 'F_pct_Com_2007', 'F_pct_Com_2008',
         'F_pct_Com_2009', 'F_pct_Com_2010', 'F_pct_Com_2011', 'F_pct_Com_2012',
         'F_pct_Com_2101', 'F_pct_Com_2102', 'F_pct_Com_2103', 'F_pct_Com_2104',
         'F_pct_Com_2105', 'F_pct_Com_2106', 'F_pct_Com_2107', 'F_pct_Com_2108',
         'F_pct_Com_2109', 'F_pct_Com_2110', 'F_pct_Com_2111', 'F_pct_Com_2112',
         'F_pct_Com_2201', 'F_pct_Com_2202', 'F_pct_Com_2203', 'F_pct_Com_2204',
         'F_pct_Com_2205', 'F_pct_Com_2206']

F_Ven = ['F_pct_Ven_1905', 'F_pct_Ven_1906', 'F_pct_Ven_1907', 'F_pct_Ven_1908', 
         'F_pct_Ven_1909', 'F_pct_Ven_1910', 'F_pct_Ven_1911', 'F_pct_Ven_1912',
         'F_pct_Ven_2001', 'F_pct_Ven_2002', 'F_pct_Ven_2003', 'F_pct_Ven_2004',
         'F_pct_Ven_2005', 'F_pct_Ven_2006', 'F_pct_Ven_2007', 'F_pct_Ven_2008',
         'F_pct_Ven_2009', 'F_pct_Ven_2010', 'F_pct_Ven_2011', 'F_pct_Ven_2012',
         'F_pct_Ven_2101', 'F_pct_Ven_2102', 'F_pct_Ven_2103', 'F_pct_Ven_2104',
         'F_pct_Ven_2105', 'F_pct_Ven_2106', 'F_pct_Ven_2107', 'F_pct_Ven_2108',
         'F_pct_Ven_2109', 'F_pct_Ven_2110', 'F_pct_Ven_2111', 'F_pct_Ven_2112',
         'F_pct_Ven_2201', 'F_pct_Ven_2202', 'F_pct_Ven_2203', 'F_pct_Ven_2204',
         'F_pct_Ven_2205', 'F_pct_Ven_2206']

Y_Com = ['Y_pct_Com_2001', 'Y_pct_Com_2002', 'Y_pct_Com_2003', 'Y_pct_Com_2004',
         'Y_pct_Com_2005', 'Y_pct_Com_2006', 'Y_pct_Com_2007', 'Y_pct_Com_2008',
         'Y_pct_Com_2009', 'Y_pct_Com_2010', 'Y_pct_Com_2011', 'Y_pct_Com_2012',
         'Y_pct_Com_2101', 'Y_pct_Com_2102', 'Y_pct_Com_2103', 'Y_pct_Com_2104',
         'Y_pct_Com_2105', 'Y_pct_Com_2106', 'Y_pct_Com_2107', 'Y_pct_Com_2108',
         'Y_pct_Com_2109', 'Y_pct_Com_2110', 'Y_pct_Com_2111', 'Y_pct_Com_2112',
         'Y_pct_Com_2201', 'Y_pct_Com_2202', 'Y_pct_Com_2203', 'Y_pct_Com_2204',
         'Y_pct_Com_2205', 'Y_pct_Com_2206']

Y_Ven = ['Y_pct_Ven_2001', 'Y_pct_Ven_2002', 'Y_pct_Ven_2003', 'Y_pct_Ven_2004',
         'Y_pct_Ven_2005', 'Y_pct_Ven_2006', 'Y_pct_Ven_2007', 'Y_pct_Ven_2008',
         'Y_pct_Ven_2009', 'Y_pct_Ven_2010', 'Y_pct_Ven_2011', 'Y_pct_Ven_2012',
         'Y_pct_Ven_2101', 'Y_pct_Ven_2102', 'Y_pct_Ven_2103', 'Y_pct_Ven_2104',
         'Y_pct_Ven_2105', 'Y_pct_Ven_2106', 'Y_pct_Ven_2107', 'Y_pct_Ven_2108',
         'Y_pct_Ven_2109', 'Y_pct_Ven_2110', 'Y_pct_Ven_2111', 'Y_pct_Ven_2112',
         'Y_pct_Ven_2201', 'Y_pct_Ven_2202', 'Y_pct_Ven_2203', 'Y_pct_Ven_2204',
         'Y_pct_Ven_2205', 'Y_pct_Ven_2206']

basics = ['Modelo', 'Subrubro']

## Para todos los Subrubros juntos

### Aplico MinMax

In [ ]:
# Crear un objeto MinMaxScaler
scaler = MinMaxScaler()
# Aplicar el escalado Min-Max solo a las columnas seleccionadas
vectores[F_Ven] = scaler.fit_transform(vectores[F_Ven])
vectores

### Aplico PCA a 12 componentes

In [ ]:
pca=decomposition.PCA(n_components=12) #elegimos 2, 3 o 4 pero pueden ser más

pca.fit(vectores[F_Ven])

# proporción de varianza
print('proporción de varianza por componente: ', pca.explained_variance_ratio_)
# proporción de varianza acumulada
print ('proporción de varianza por componente acumulada: ', pca.explained_variance_ratio_.cumsum())

vectores_pca=pca.transform(vectores[F_Ven]) #numpy array
vectores_pca = pd.DataFrame(data=vectores_pca, columns=[f'PC{i}' for i in range(1, 13)]).copy()

print ('tamaño de los datos: ', vectores_pca.shape)
vectores_pca

### Elijo el nro de clusters con el método del codo

In [ ]:
#Prueba: para elegir el hiperparámetro n_clusters, variando de 2 a 11 clusters
scores = [KMeans(n_clusters=i, n_init=10).fit(vectores_pca).inertia_ for i in range(2,12)]

plt.plot(np.arange(2, 12), scores)
plt.xlabel('Number of clusters')
plt.ylabel("Inertia")
plt.title("Inertia of k-Means versus number of clusters")

In [ ]:
# Encontrar el número recomendado de clusters (punto del codo)
# Encuentra la primera derivada (diferencia entre puntos sucesivos)
differences = np.diff(scores)

# Encuentra el punto del codo (donde la primera derivada cambia más)
recommended_num_clusters = np.where(differences == max(differences))[0][0] + 2  # +2 para compensar el rango de inicio

print("Número recomendado de clusters:", recommended_num_clusters)

### Aplico KMeans

In [ ]:
km = KMeans(n_clusters=recommended_num_clusters, n_init=10) # El parámetro n_init igual a 10 me lo pide para no tirar error. Ver luego qué implica.
km.fit(vectores_pca)
clusters = km.labels_

### Aplico PCA en dos componentes para graficar

In [ ]:
pca=decomposition.PCA(n_components=2) #elegimos 2, 3 o 4 pero pueden ser más

pca.fit(vectores_pca)

# proporción de varianza
print('proporción de varianza por componente: ', pca.explained_variance_ratio_)
# proporción de varianza acumulada
print ('proporción de varianza por componente acumulada: ', pca.explained_variance_ratio_.cumsum())

vectores_pca_2=pca.transform(vectores_pca) #numpy array
vectores_pca_2 = pd.DataFrame(data=vectores_pca_2, columns=[f'PC{i}' for i in range(1, 3)]).copy()

print ('tamaño de los datos: ', vectores_pca_2.shape)
vectores_pca_2

### Agrego etiquetas y otras variables para graficar

In [ ]:
vectores_clusters = vectores_pca_2.copy()
vectores_clusters['kmeans_n'] = km.labels_
print('Kmeans encontró: ', max(km.labels_)+1, 'clusters, nosotros forzamos la cantidad')
vectores_clusters

In [ ]:
vectores_clusters['kmeans_n'].value_counts()

In [ ]:
vectores_clusters = pd.concat([vectores[basics], vectores_clusters], axis=1)
vectores_clusters

### Grafico

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Supongamos que vectores_clusters contiene tus datos

# Crear una figura con dos subtramas en una fila
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
axs[0].set_title("Vista normal")
axs[1].set_title("Zoom en (0, 0)")

# Configurar el pairplots en ambas subtramas
for ax in axs:
    sns.scatterplot(data=vectores_clusters, x='PC1', y='PC2', hue='kmeans_n', palette='Set1', s=20, ax=ax)

# Filtrar los valores donde Modelo == 1
filt_modelo_1 = vectores_clusters['Modelo'] == 1

# Personalizar el tamaño y color de los puntos donde Modelo == 1
for ax in axs:
    sns.scatterplot(data=vectores_clusters[filt_modelo_1], x='PC1', y='PC2', color='black', s=20, label='Modelo', ax=ax)

# Establecer los límites de los ejes en la segunda subtrama (Zoom)
axs[1].set_xlim(-0.15, 0.15)
axs[1].set_ylim(-0.15, 0.15)

# Eliminar las leyendas de las subtramas individuales
for ax in axs:
    ax.get_legend().remove()

# Agregar una única leyenda fuera de las subtramas
handles, labels = axs[1].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper left", bbox_to_anchor=(1, 1))

# Mostrar el gráfico completo
plt.tight_layout()
plt.show()

## Por subrubro

In [ ]:
vectores = pd.read_csv('../data/interim/tp3_Fven_limpio_ceros.csv')

# Identificar subrubros
unique_subrubros = vectores['Subrubro'].unique()

# Itera sobre los subrubros únicos y crea gráficos para cada uno
for subrubro in unique_subrubros:
    subrubro_df = vectores[vectores['Subrubro'] == subrubro].copy()
    
    # Escala MinMax
    scaler = MinMaxScaler()
    subrubro_df[F_Ven] = scaler.fit_transform(subrubro_df[F_Ven])

    # PCA de 12 componentes
    pca = decomposition.PCA(n_components=12)
    pca.fit(subrubro_df[F_Ven])
    subrubro_df_pca = pca.transform(subrubro_df[F_Ven])
    subrubro_df_pca = pd.DataFrame(data=subrubro_df_pca, columns=[f'PC{i}' for i in range(1, 13)]).copy()

    # Nro de clusters recomendado por el codo
    scores = [KMeans(n_clusters=i, n_init=10).fit(subrubro_df_pca).inertia_ for i in range(2, 12)]
    differences = np.diff(scores)
    recommended_num_clusters = np.where(differences == max(differences))[0][0] + 2

    # KMeans
    km = KMeans(n_clusters=recommended_num_clusters, n_init=10)
    km.fit(subrubro_df_pca)
    clusters = km.labels_

    # PCA de 2 componentes para visualizar
    pca = decomposition.PCA(n_components=2)
    pca.fit(subrubro_df_pca)
    subrubro_df_pca_2 = pca.transform(subrubro_df_pca)  # numpy array
    subrubro_df_pca_2 = pd.DataFrame(data=subrubro_df_pca_2, columns=[f'PC{i}' for i in range(1, 3)]).copy()
    subrubro_df_clusters = subrubro_df_pca_2.copy()
    subrubro_df_clusters['kmeans_n'] = km.labels_
    subrubro_df_clusters = pd.concat([subrubro_df[basics], subrubro_df_clusters], axis=1)

    # Gráfico
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    fig.suptitle(f"Subrubro: {subrubro}", fontsize=16)
    axs[0].set_title("Vista normal")
    axs[1].set_title("Zoom en (0, 0)")
    for ax in axs:
        sns.scatterplot(data=subrubro_df_clusters, x='PC1', y='PC2', hue='kmeans_n', palette='Set1', s=20, ax=ax)
    filt_modelo_1 = subrubro_df_clusters['Modelo'] == 1
    for ax in axs:
        sns.scatterplot(data=subrubro_df_clusters[filt_modelo_1], x='PC1', y='PC2', color='black', s=20, label='Modelo', ax=ax)
    axs[1].set_xlim(-0.15, 0.15)
    axs[1].set_ylim(-0.15, 0.15)
    for ax in axs:
        ax.get_legend().remove()
    handles, labels = axs[1].get_legend_handles_labels()
    fig.legend(handles, labels, loc="upper left", bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()

#### Fuerzo que elija 2 clusters

In [ ]:
vectores = pd.read_csv('../data/interim/tp3_Fven_limpio_ceros.csv')

# Identificar subrubros
unique_subrubros = vectores['Subrubro'].unique()

# Itera sobre los subrubros únicos y crea gráficos para cada uno
for subrubro in unique_subrubros:
    subrubro_df = vectores[vectores['Subrubro'] == subrubro].copy()
    
    # Escala MinMax
    scaler = MinMaxScaler()
    subrubro_df[F_Ven] = scaler.fit_transform(subrubro_df[F_Ven])

    # PCA de 12 componentes
    pca = decomposition.PCA(n_components=12)
    pca.fit(subrubro_df[F_Ven])
    subrubro_df_pca = pca.transform(subrubro_df[F_Ven])
    subrubro_df_pca = pd.DataFrame(data=subrubro_df_pca, columns=[f'PC{i}' for i in range(1, 13)]).copy()

    # Nro de clusters elegido
    recommended_num_clusters = 2

    # KMeans
    km = KMeans(n_clusters=recommended_num_clusters, n_init=10)
    km.fit(subrubro_df_pca)
    clusters = km.labels_

    # PCA de 2 componentes para visualizar
    pca = decomposition.PCA(n_components=2)
    pca.fit(subrubro_df_pca)
    subrubro_df_pca_2 = pca.transform(subrubro_df_pca)  # numpy array
    subrubro_df_pca_2 = pd.DataFrame(data=subrubro_df_pca_2, columns=[f'PC{i}' for i in range(1, 3)]).copy()
    subrubro_df_clusters = subrubro_df_pca_2.copy()
    subrubro_df_clusters['kmeans_n'] = km.labels_
    subrubro_df_clusters = pd.concat([subrubro_df[basics], subrubro_df_clusters], axis=1)

    # Gráfico
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    fig.suptitle(f"Subrubro: {subrubro}", fontsize=16)
    axs[0].set_title("Vista normal")
    axs[1].set_title("Zoom en (0, 0)")
    for ax in axs:
        sns.scatterplot(data=subrubro_df_clusters, x='PC1', y='PC2', hue='kmeans_n', palette='Set1', s=20, ax=ax)
    filt_modelo_1 = subrubro_df_clusters['Modelo'] == 1
    for ax in axs:
        sns.scatterplot(data=subrubro_df_clusters[filt_modelo_1], x='PC1', y='PC2', color='black', s=20, label='Modelo', ax=ax)
    axs[1].set_xlim(-0.15, 0.15)
    axs[1].set_ylim(-0.15, 0.15)
    for ax in axs:
        ax.get_legend().remove()
    handles, labels = axs[1].get_legend_handles_labels()
    fig.legend(handles, labels, loc="upper left", bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()

In [ ]:
#vectores_interes = vectores[basics + Y_Ven]
vectores_interes = vectores[Y_Ven]
vectores_interes.head()

In [ ]:
vectores_interes.info()

Hay problemas con los vacíos, para que me los tome el modelo:

In [ ]:
# Ventas anuales
vectores_interes = vectores[Y_Ven]
msno.matrix(vectores_interes, fontsize=12, color=[0.5,0,0], figsize=(6, 5))
plt.show()

In [ ]:
# Ventas anuales(zoom)
vectores_interes = vectores[Y_Ven]
msno.matrix(vectores_interes[185:190], fontsize=12, color=[0.5,0,0], figsize=(6, 5))
plt.show()

In [ ]:
vectores.iloc[186, :]

In [ ]:
vectores['Y_pct_Ven_2109'].max()

In [ ]:
vectores[vectores['Y_pct_Ven_2109'] == 203697461.3233292]

In [ ]:
ventas_fisc = pd.read_csv('../data/interim/tp2_ventas_fisc.csv')
ventas_fisc[(ventas_fisc['ID'] == 186) & (ventas_fisc['Subrubro'] == 'Com. Varios')]

In [ ]:
ventas_fisc[(ventas_fisc['ID'] == 186)]['Subrubro'].value_counts()

**HACER**
- Llevar a cero todo lo que está entre -1000 y +1000 (esto antes del tratamiento de inflación)

20/9 >> 2.287884e-02
21/9 >> 4.660361e+06
3017	1	Gondola de dónde viene para dar ese 10-2?



In [ ]:
ventas_fisc[ventas_fisc['ID'] == 3017]

In [ ]:
# Ventas cuatrimestrales
vectores_interes = vectores[F_Ven]
msno.matrix(vectores_interes, fontsize=12, color=[0.5,0,0], figsize=(6, 5))
plt.show()

In [ ]:
# Comisiones anuales
vectores_interes = vectores[Y_Com]
msno.matrix(vectores_interes, fontsize=12, color=[0.5,0,0], figsize=(6, 5))
plt.show()

In [ ]:
# Comisiones cuatrimestrales
vectores_interes = vectores[F_Com]
msno.matrix(vectores_interes, fontsize=12, color=[0.5,0,0], figsize=(6, 5))
plt.show()

In [ ]:
msno.bar(vectores_interes, fontsize=12, color="tab:blue", figsize=(10, 4))
plt.show()

In [ ]:
# Paso 1: Contar valores vacíos por fila
vacios_por_fila = vectores_interes.isnull().sum(axis=1)

# Paso 2: Generar un resumen de cuántas filas tienen 0, 1, 2, 3, etc., valores vacíos
resumen = vacios_por_fila.value_counts().reset_index()
resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')
resumen.head()

# ** Chequear **  >> Esto borra teniendo en cuenta yearly

In [ ]:
plt.figure(figsize=(10, 4))  # Tamaño del gráfico
plt.bar(resumen['Cantidad de Valores Vacíos'], resumen['Número de Filas'], color='blue', alpha=0.7)
plt.xlabel('Cantidad de Valores Vacíos por Vendedor')
plt.ylabel('Frecuencia')
plt.title('Distribución de Valores Vacíos por Vendedor')
plt.xticks(resumen['Cantidad de Valores Vacíos'])
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Muestra el gráfico
plt.show()


También hay problema de outliers. Qué significa que una variación me dé -20.000%. Que tenía ej 1.000.000 y pasé a 0,05?. Algo hay que hacer

In [ ]:
vectores_interes.describe()

In [ ]:
# # Configuración del diseño del gráfico
# plt.figure(figsize=(16, 10))  # Tamaño del layout

# # Itera a través de cada columna y crea un KDE plot
# for column in vectores_interes.columns:
#     plt.subplot(5, 7, vectores_interes.columns.get_loc(column) + 1)  # 5 filas, 7 columnas
#     sns.kdeplot(vectores_interes[column], fill=True)
#     plt.title(column)

# # Ajusta la disposición y muestra el gráfico
# plt.tight_layout()
# plt.show()

In [ ]:
# Configuración del diseño del gráfico
plt.figure(figsize=(16, 10))  # Tamaño del layout

# Itera a través de cada columna y crea un boxplot
for i, column in enumerate(vectores_interes.columns):
    plt.subplot(5, 7, i + 1)  # 5 filas, 7 columnas
    sns.boxplot(x=vectores_interes[column])
    plt.title(column)

# Ajusta la disposición y muestra el gráfico
plt.tight_layout()
plt.show()

---
# Selección del Número de Clusters <span style="color:magenta">**(Paso 2)**</span>

### Elbow method

Para probar, le quito los vacíos.

In [ ]:
vectores_interes_sin_vacios.describe()

In [ ]:
gondola = vectores[vectores['Subrubro'] == 'Gondola']

#[basics + Y_Ven]
vectores_interes = gondola[Y_Ven]

In [ ]:
vectores_interes_sin_vacios = vectores_interes.dropna().copy()
print(vectores_interes_sin_vacios.shape)

# me quedo con el +-500% que se ve como un 5 acá OJO
vec_pm500 = vectores_interes_sin_vacios.where((vectores_interes_sin_vacios < 5) & (vectores_interes_sin_vacios > - 5))
vec_pm500 = vec_pm500.dropna()

vectores_interes_sin_vacios = vec_pm500

In [ ]:
vec_pm500.describe()

In [ ]:
#Prueba: para elegir el hiperparámetro n_clusters, variando de 2 a 11 clusters
scores = [KMeans(n_clusters=i, n_init=10).fit(vectores_interes_sin_vacios).inertia_ for i in range(2,12)]

plt.plot(np.arange(2, 12), scores)
plt.xlabel('Number of clusters')
plt.ylabel("Inertia")
plt.title("Inertia of k-Means versus number of clusters")

### Coeficiente Silhouette

---
## Aplicación de Modelos de Clustering <span style="color:magenta">**(Paso 3)**</span>

### K-means

In [ ]:
km = KMeans(n_clusters=3, n_init=10) # El parámetro n_init igual a 10 me lo pide para no tirar error. Ver luego qué implica.
km.fit(vectores_interes_sin_vacios)
clusters = km.labels_

In [ ]:
vectores_clusters = vectores_interes_sin_vacios.copy()
vectores_clusters['kmeans_4'] = km.labels_
print('Kmeans encontró: ', max(km.labels_)+1, 'clusters, nosotros forzamos la cantidad')
vectores_clusters.head(4)

In [ ]:
# Esto está hecho metiendo a todos los rubros en la misma bolsa
# Hacer por subrubro
vectores_clusters['kmeans_4'].value_counts()

In [ ]:
std_scale=preprocessing.StandardScaler().fit(vec_pm500)
X_scaled=std_scale.transform(vec_pm500) # numpyarray Estandarizado (le resta la media y divide por el desvío) por columna

pca=decomposition.PCA(n_components=4) #elegimos 2, 3 o 4 pero pueden ser más,

pca.fit(X_scaled) #input data is centered but not scaled for each feature before applying the SVD

# proporción de varianza
print('proporción de varianza por componente: ', pca.explained_variance_ratio_)
# proporción de varianza acumulada
print ('proporción de varianza por componente acumulada: ', pca.explained_variance_ratio_.cumsum())

X_projected=pca.transform(X_scaled) #numpy array
print ('tamaño de los datos: ', X_projected.shape)


In [ ]:
X_tsne_kmeans_4 = pd.DataFrame(X_projected)
X_tsne_kmeans_4['kmeans_4'] = km.labels_

sns.scatterplot(data=X_tsne_kmeans_4,
                x=0, 
                y=1, 
                hue="kmeans_4", 
                palette="deep")#, alpha=0.25)

# sns.pairplot(X_projected)

In [ ]:
from sklearn import (manifold, preprocessing, decomposition)

tsne = manifold.TSNE(n_components=2, verbose=1,perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(vec_pm500)

In [ ]:
X_tsne_kmeans_4 = pd.DataFrame(X_tsne)
X_tsne_kmeans_4['kmeans_4'] = km.labels_

sns.scatterplot(data=X_tsne_kmeans_4,
                x=0, 
                y=1, 
                hue="kmeans_4", 
                palette="deep")

---


In [ ]:
gondola = vectores[vectores['Subrubro'] == 'Gondola']

#[basics + Y_Ven]
vectores_interes = gondola[F_Ven]

In [ ]:
vectores_interes_sin_vacios = vectores_interes.dropna().copy()
print(vectores_interes_sin_vacios.shape)

# me quedo con el +-500% que se ve como un 5 acá OJO
vec_pm500 = vectores_interes_sin_vacios.where((vectores_interes_sin_vacios < 5) & (vectores_interes_sin_vacios > - 5))
vec_pm500 = vec_pm500.dropna()

vectores_interes_sin_vacios = vec_pm500

In [ ]:
vec_pm500.describe()

In [ ]:
#Prueba: para elegir el hiperparámetro n_clusters, variando de 2 a 11 clusters
scores = [KMeans(n_clusters=i, n_init=10).fit(vectores_interes_sin_vacios).inertia_ for i in range(2,12)]

plt.plot(np.arange(2, 12), scores)
plt.xlabel('Number of clusters')
plt.ylabel("Inertia")
plt.title("Inertia of k-Means versus number of clusters")

### Coeficiente Silhouette

---
## Aplicación de Modelos de Clustering <span style="color:magenta">**(Paso 3)**</span>

### K-means

In [ ]:
km = KMeans(n_clusters=3, n_init=10) # El parámetro n_init igual a 10 me lo pide para no tirar error. Ver luego qué implica.
km.fit(vectores_interes_sin_vacios)
clusters = km.labels_

In [ ]:
vectores_clusters = vectores_interes_sin_vacios.copy()
vectores_clusters['kmeans_4'] = km.labels_
print('Kmeans encontró: ', max(km.labels_)+1, 'clusters, nosotros forzamos la cantidad')
vectores_clusters.head(4)

In [ ]:
# Esto está hecho metiendo a todos los rubros en la misma bolsa
# Hacer por subrubro
vectores_clusters['kmeans_4'].value_counts()

In [ ]:
pca=decomposition.PCA(n_components=2) #elegimos 2, 3 o 4 pero pueden ser más,

pca.fit(vec_pm500) #input data is centered but not scaled for each feature before applying the SVD

# proporción de varianza
print('proporción de varianza por componente: ', pca.explained_variance_ratio_)
# proporción de varianza acumulada
print ('proporción de varianza por componente acumulada: ', pca.explained_variance_ratio_.cumsum())

X_projected=pca.transform(vec_pm500) #numpy array
print ('tamaño de los datos: ', X_projected.shape)


In [ ]:
X_tsne_kmeans_4 = pd.DataFrame(X_projected)
X_tsne_kmeans_4['kmeans_4'] = km.labels_

sns.scatterplot(data=X_tsne_kmeans_4,
                x=0, 
                y=1, 
                hue="kmeans_4", 
                palette="deep")#, alpha=0.25)

# sns.pairplot(X_projected)

In [ ]:
from sklearn import (manifold, preprocessing, decomposition)

tsne = manifold.TSNE(n_components=2, verbose=1,perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(vec_pm500)

In [ ]:
X_tsne_kmeans_4 = pd.DataFrame(X_tsne)
X_tsne_kmeans_4['kmeans_4'] = km.labels_

sns.scatterplot(data=X_tsne_kmeans_4,
                x=0, 
                y=1, 
                hue="kmeans_4", 
                palette="deep")

In [ ]:
selected_columns = [Y_Ven[0], Y_Ven[1], 'kmeans_4']
sns.pairplot(vectores_clusters[selected_columns], hue='kmeans_4')

Probando para un solo rubro..

In [ ]:
vectores_interes = vectores[basics + Y_Ven]
vectores_gondola = vectores_interes[vectores_interes['Subrubro'] == 'Gondola'].copy()
vectores_gondola = vectores_gondola.dropna().copy()
vectores_gondola

In [ ]:
vectores_gondola = vectores_gondola[Y_Ven].copy()

In [ ]:
#Prueba: para elegir el hiperparámetro n_clusters, variando de 2 a 11 clusters
scores = [KMeans(n_clusters=i, n_init=10).fit(vectores_gondola).inertia_ for i in range(2,12)]

plt.plot(np.arange(2, 12), scores)
plt.xlabel('Number of clusters')
plt.ylabel("Inertia")
plt.title("Inertia of k-Means versus number of clusters")

In [ ]:
km = KMeans(n_clusters=4, n_init=10) # El parámetro n_init igual a 10 me lo pide para no tirar error. Ver luego qué implica.
km.fit(vectores_gondola)
clusters = km.labels_

In [ ]:
vectores_clusters = vectores_gondola.copy()
vectores_clusters['kmeans_4'] = km.labels_
print('Kmeans encontró: ', max(km.labels_)+1, 'clusters, nosotros forzamos la cantidad')
vectores_clusters.head(4)

In [ ]:
vectores_clusters['kmeans_4'].value_counts()

In [ ]:
selected_columns = [Y_Ven[0], Y_Ven[1], 'kmeans_4']
sns.pairplot(vectores_clusters[selected_columns], hue='kmeans_4')

### Otros algoritmos de clustering

---
## Visualización de Resultados <span style="color:magenta">**(Paso 4)**</span>

---
## Interpretación y Evaluación <span style="color:magenta">**(Paso 5)**</span>

---
## Confianza en los Resultados <span style="color:magenta">**(Paso 6)**</span>

---
## Preguntas finales <span style="color:magenta">**(Paso 7)**</span>